<a href="https://colab.research.google.com/github/crdhilep/Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

# import pyspark
import pyspark

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Hit:13 

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Vine_Review_Analysis").getOrCreate()

In [3]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Baby_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Baby_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    9970739| R8EWA1OFT84NX|B00GSP5D94|     329991347|Summer Infant Swa...|            Baby|          5|            0|          0|   N|                Y|Great swaddled bl...|Loved these swadd...| 2015-08-31|
|         US|   23538442|R2JWY4YRQD4FOP|B00YYDDZGU|     646108902|Pacifier Clip Gir...|            Baby|          5|    

In [4]:
# filter DataFrame for total_votes above or equal to 20
filtered_df = df.filter(df.total_votes >= 20)
filtered_df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   14499497| ROI00QN8IS49I|B00M5CMRNE|     326338622|Baltic Amber Teet...|            Baby|          5|           83|         91|   N|                N|Absolutely GORGEO...|Beautiful Amber!<...| 2015-08-31|
|         US|    1243530|R13C5INE1RTZP6|B00G66UDAO|     130471439|Sportspower For B...|            Baby|          2|    

In [5]:
# Filter DataFrame for helpful_votes ratio above or equal to 50%
helpful_votes = filtered_df.filter((filtered_df.helpful_votes / filtered_df.total_votes) >= 0.5)
helpful_votes.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   14499497| ROI00QN8IS49I|B00M5CMRNE|     326338622|Baltic Amber Teet...|            Baby|          5|           83|         91|   N|                N|Absolutely GORGEO...|Beautiful Amber!<...| 2015-08-31|
|         US|    1243530|R13C5INE1RTZP6|B00G66UDAO|     130471439|Sportspower For B...|            Baby|          2|    

In [7]:
# Create paid vine DataFrame
paid_vine = helpful_votes.filter(helpful_votes.vine == 'Y')
paid_vine.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   21155796|R3N0XV9267NOXV|B00ZKTVI66|     155258416|Graco LiteRider C...|            Baby|          5|           41|         47|   Y|                N|No more disturbin...|This version of t...| 2015-08-31|
|         US|   25087234| RSA6JQ346JZHZ|B00SIL7EZ2|     916919021|Skip Hop Moonligh...|            Baby|          5|    

In [8]:
# Create unpaid vine DataFrame
unpaid_vine = helpful_votes.filter(helpful_votes.vine == 'N')
unpaid_vine.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   14499497| ROI00QN8IS49I|B00M5CMRNE|     326338622|Baltic Amber Teet...|            Baby|          5|           83|         91|   N|                N|Absolutely GORGEO...|Beautiful Amber!<...| 2015-08-31|
|         US|    1243530|R13C5INE1RTZP6|B00G66UDAO|     130471439|Sportspower For B...|            Baby|          2|    

In [16]:
# paid total number of reviews
total_paid_reviews = paid_vine.count()
total_paid_reviews

463

In [10]:
# paid 5-star reviews
paid_five_star_reviews = paid_vine.filter(paid_vine.star_rating == 5).count()
paid_five_star_reviews

202

In [11]:
# paid 5-star reviews percentage
paid_five_star_percent = (paid_five_star_reviews / total_paid_reviews) * 100
paid_five_star_percent

43.628509719222464

In [12]:
# unpaid total number of reviews
total_unpaid_reviews = unpaid_vine.count()
total_unpaid_reviews

25094

In [13]:
# unpaid 5-star reviews
unpaid_five_star_reviews = unpaid_vine.filter(unpaid_vine.star_rating == 5).count()
unpaid_five_star_reviews

12033

In [14]:
# unpaid 5-star reviews percentage
unpaid_five_star_percent = (unpaid_five_star_reviews / total_unpaid_reviews) * 100
unpaid_five_star_percent

47.95170160197657